# Milestone 2 

#### Initial analysis and data handling pipeline for the project. 

- Load data
- Data wrangling 
    - missing data
    - incorrect data / corrupted data
    - parse/transform data into appropriate data model and format (for the analysis we want to do)
- Data analysis
    - descriptive statistics
    - initial analysis
    
Goal is to prove that project idea is feasible given the data :)